In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata_large.csv')

In [3]:
dataset.describe()

,Age,Salary,Purchased
count,1554.000000,1554.000000,1554.000000
mean,44.296010,57042.471042,0.694981
std,17.462458,21209.244800,0.460564
min,18.000000,20000.000000,0.000000
25%,27.000000,46000.000000,0.000000
50%,43.000000,60000.000000,1.000000
75%,62.000000,66000.000000,1.000000
max,69.000000,96000.000000,1.000000


In [4]:
dataset.head()

,Age,Salary,Purchased
0,18,20000,0
1,19,22000,0
2,20,24000,0
3,21,28000,0
4,22,60000,1


In [5]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:,-1].values

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
# converting array to torch array
Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(X_test).float()

In [9]:
Xtrain_

tensor([[-0.0935,  0.2215],
        [ 1.2217, -0.5342],
        [-1.0655,  0.4104],
        ...,
        [ 1.2789,  0.4104],
        [-0.9512,  0.2215],
        [-1.2943, -1.4316]])

In [10]:
ytrain_ = torch.from_numpy(y_train)
ytest_ = torch.from_numpy(y_test)

In [11]:
ytrain_

tensor([1, 1, 1,  ..., 1, 1, 0])

In [12]:
Xtrain_.shape, ytrain_.shape

(torch.Size([1243, 2]), torch.Size([1243]))

In [13]:
Xtest_.shape, ytest_.shape

(torch.Size([311, 2]), torch.Size([311]))

In [14]:
input_size=2 # input features
output_size=2 # output features
hidden_size=10 # hidden neurons

In [15]:
# defining the PyTorch model
class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.fc1 = torch.nn.Linear(input_size, hidden_size)
       self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
       self.fc3 = torch.nn.Linear(hidden_size, output_size)


   def forward(self, X):
       X = torch.relu((self.fc1(X)))
       X = torch.relu((self.fc2(X)))
       X = self.fc3(X)

       return F.log_softmax(X,dim=1)

In [17]:
model = Net()


In [18]:
import torch.optim as optim
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()

In [19]:
epochs = 100

In [20]:
# model training
for epoch in range(epochs):
  optimizer.zero_grad()
  Ypred = model(Xtrain_)
  loss = loss_fn(Ypred,  ytrain_)
  loss.backward()
  optimizer.step()
  print('Epoch',epoch, 'loss',loss.item())

Epoch 0 loss 0.6752635836601257
Epoch 1 loss 0.6460391283035278
Epoch 2 loss 0.6196292042732239
Epoch 3 loss 0.5945513248443604
Epoch 4 loss 0.5708674192428589
Epoch 5 loss 0.5484994649887085
Epoch 6 loss 0.5273096561431885
Epoch 7 loss 0.5072876811027527
Epoch 8 loss 0.488503634929657
Epoch 9 loss 0.4710187613964081
Epoch 10 loss 0.45452386140823364
Epoch 11 loss 0.43906712532043457
Epoch 12 loss 0.42424893379211426
Epoch 13 loss 0.40970751643180847
Epoch 14 loss 0.3951682448387146
Epoch 15 loss 0.380665123462677
Epoch 16 loss 0.3662615120410919
Epoch 17 loss 0.3522205948829651
Epoch 18 loss 0.33839213848114014
Epoch 19 loss 0.32491010427474976
Epoch 20 loss 0.3117380738258362
Epoch 21 loss 0.2987034022808075
Epoch 22 loss 0.28584960103034973
Epoch 23 loss 0.2731923758983612
Epoch 24 loss 0.26079660654067993
Epoch 25 loss 0.24862590432167053
Epoch 26 loss 0.23667852580547333
Epoch 27 loss 0.22497592866420746
Epoch 28 loss 0.21367421746253967
Epoch 29 loss 0.20321036875247955
Epoch 30 

In [21]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.6406, -0.3251],
         [-0.1492,  0.1595],
         [ 1.1633,  0.5958],
         [-0.4106,  0.4027],
         [ 1.0447,  0.8815],
         [-0.0014,  0.6626],
         [-0.7802, -0.8736],
         [ 0.9894, -0.1063],
         [-0.1443,  1.0242],
         [ 1.0917, -0.1393]], requires_grad=True),
 Parameter containing:
 tensor([-0.4538, -0.5055,  0.8142,  0.3233,  0.6768,  0.5435,  1.0104,  0.2746,
          0.5999,  0.0880], requires_grad=True),
 Parameter containing:
 tensor([[ 0.2719, -0.0598,  0.6311,  0.3459,  0.2693,  0.6105, -0.1772,  0.2152,
           0.6075,  0.6806],
         [ 0.4362, -0.1592,  0.3940,  0.5623,  0.4358,  0.6848, -0.3512,  0.6086,
           0.4168,  0.6284],
         [-0.0107,  0.0150,  0.2185,  0.1446, -0.2910, -0.2974, -0.1164, -0.1584,
           0.1499,  0.2407],
         [ 0.0989, -0.0779, -0.0687,  0.0594, -0.1168,  0.1300, -0.3991, -0.3156,
           0.0603, -0.1055],
         [-0.3312, -0.0187,  0.2128, -0.2756,

In [22]:
torch.from_numpy(sc.transform(np.array([[40,20000]]))).float()

tensor([[-0.2650, -1.7622]])

In [23]:
y_cust_20_40000 = model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.0073, -4.9168]], grad_fn=<LogSoftmaxBackward0>)

In [24]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000

tensor([0])

In [25]:
y_cust_42_50000 = model(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000

tensor([[-1.0255, -0.4441]], grad_fn=<LogSoftmaxBackward0>)

In [26]:
_, predicted_42_50000 = torch.max(y_cust_42_50000.data,-1)
predicted_42_50000

tensor([1])

In [27]:
# saving the model
torch.save(model,'customer_buy.pt')

In [28]:
!ls

customer_buy.pt  sample_data


In [29]:
restored_model = torch.load('customer_buy.pt')

In [30]:
y_cust_20_40000 = restored_model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.0073, -4.9168]], grad_fn=<LogSoftmaxBackward0>)

In [31]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000

tensor([0])

In [32]:
model.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.6406, -0.3251],
                      [-0.1492,  0.1595],
                      [ 1.1633,  0.5958],
                      [-0.4106,  0.4027],
                      [ 1.0447,  0.8815],
                      [-0.0014,  0.6626],
                      [-0.7802, -0.8736],
                      [ 0.9894, -0.1063],
                      [-0.1443,  1.0242],
                      [ 1.0917, -0.1393]])),
             ('fc1.bias',
              tensor([-0.4538, -0.5055,  0.8142,  0.3233,  0.6768,  0.5435,  1.0104,  0.2746,
                       0.5999,  0.0880])),
             ('fc2.weight',
              tensor([[ 0.2719, -0.0598,  0.6311,  0.3459,  0.2693,  0.6105, -0.1772,  0.2152,
                        0.6075,  0.6806],
                      [ 0.4362, -0.1592,  0.3940,  0.5623,  0.4358,  0.6848, -0.3512,  0.6086,
                        0.4168,  0.6284],
                      [-0.0107,  0.0150,  0.2185,  0.1446, -0.2910, -0.2974, -0.1164

In [33]:
# this should be the recommended approach to save the model
torch.save(model.state_dict(),'customer_buy_state_dict')

In [34]:
!ls

customer_buy.pt  customer_buy_state_dict  sample_data


In [35]:
new_predictor = Net()

In [36]:
y_cust_20_40000 = new_predictor(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-1.1578, -0.3771]], grad_fn=<LogSoftmaxBackward0>)

In [37]:
# saving all files to a zip folder
!zip -r customer_buy_state_dict.zip customer_buy_state_dict

  adding: customer_buy_state_dict (deflated 50%)


In [38]:
!ls

customer_buy.pt		 customer_buy_state_dict.zip
customer_buy_state_dict  sample_data


In [ ]:
from google.colab import files


In [ ]:
files.download('customer_buy_state_dict.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Now loading the saved model in dictionary format and do prediction


In [39]:
# before doing it here, we need to import necessary libraries and define the necessary parameters
new_prediction2 = Net()

In [40]:
new_prediction2.load_state_dict(torch.load('customer_buy_state_dict'))

<All keys matched successfully>

In [42]:
new_prediction2(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())

tensor([[-0.0073, -4.9168]], grad_fn=<LogSoftmaxBackward0>)